# 🎨 Feature Visualization - Generación de Imágenes que Activan Neuronas

## ¿Qué hace este notebook?

Genera **imágenes sintéticas** que maximizan la activación de neuronas específicas mediante:

- 🔬 **Gradient Ascent** en el espacio de píxeles
- 🎯 **Optimización dirigida** hacia neuronas objetivo
- 🛡️ **Regularizaciones** para imágenes naturales (L2, Total Variation)
- 🔄 **Transformaciones robustas** (jitter, rotación, escala)

## ¿Por qué es útil?

A diferencia del análisis de imágenes reales (notebook 00), aquí **creamos desde cero** las imágenes que "emocionan" a cada neurona. Esto nos permite:

- Ver qué patrones busca cada filtro sin depender de imágenes existentes
- Entender qué hace que una neurona se active al máximo
- Comparar qué diferencias hay entre capas superficiales y profundas

## Estructura del notebook

1. **Setup**: Cargar modelo y herramientas
2. **Ejemplo básico**: Generar feature de una neurona
3. **Análisis de convergencia**: Ver cómo evoluciona la optimización
4. **Grid de features**: Visualizar múltiples neuronas
5. **Comparación real vs sintética**: ¿Qué activa más?
6. **Comparación entre capas**: Diferencias entre conv1 y capas profundas

---

In [ ]:
# ===================================================================
# CELDA 1: Imports y Setup
# ===================================================================
"""
Configuración inicial:
- Imports de librerías
- Configuración de device
- Carga de módulos custom
"""

import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import torch.nn as nn
from PIL import Image

# Añadir src al path
sys.path.append('..')

# Imports custom
from src.models.model_loader import ModelLoader
from src.utils.feature_visualizer import FeatureVisualizer, compare_real_vs_synthetic
from src.utils.visualization_helpers import (
    plot_feature_grid,
    plot_convergence,
    plot_real_vs_synthetic,
    plot_optimization_progress,
    find_active_neurons
)

# Configuración de matplotlib
plt.style.use('default')
%matplotlib inline

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")

# Cargar modelo
print("\n📥 Cargando modelo...")
MODEL_NAME = 'alexnet'  # Opciones: 'alexnet', 'resnet18'

loader = ModelLoader(MODEL_NAME, pretrained=True, device=device)
model = loader.load_model()

print(f"✅ Modelo {MODEL_NAME} cargado")
print(f"\n💡 TIP: Cambiar MODEL_NAME para experimentar con otras arquitecturas")

## 📚 Conceptos Teóricos

### Feature Visualization

**Objetivo**: Encontrar la imagen $x^*$ que maximiza la activación de una neurona $n_i$ en capa $l$:

$$
x^* = \arg\max_x a_i^l(x)
$$

### Proceso de Optimización

1. **Inicialización**: Empezar con ruido aleatorio
2. **Forward pass**: Calcular activación de neurona objetivo
3. **Backward pass**: Calcular gradiente respecto a píxeles
4. **Update**: Modificar píxeles en dirección del gradiente
5. **Regularización**: Aplicar penalizaciones para imágenes naturales

### Regularizaciones

**Sin regularización** → Imágenes adversariales (ruido sin sentido)

**Con regularización** → Imágenes más interpretables:

- **L2 Decay**: $\lambda_1 \|x\|^2$ → Penaliza valores extremos
- **Total Variation**: $\lambda_2 \sum |x_{i,j} - x_{i+1,j}| + |x_{i,j} - x_{i,j+1}|$ → Suaviza la imagen
- **Jitter/Rotation/Scale**: Transformaciones aleatorias → Robustez

---

In [ ]:
# ===================================================================
# DEBUG: Ver todas las capas del modelo
# ===================================================================

print("🔍 TODAS LAS CAPAS DEL MODELO:\n")

for name, module in model.named_modules():
    print(f"  '{name}'")
    if len(name) > 0 and isinstance(module, torch.nn.Conv2d):
        print(f"    → Conv2d con {module.out_channels} canales ✓")

print("\n" + "="*70)

In [ ]:
# ===================================================================
# CELDA 2: Ejemplo Básico - Generar Feature de una Neurona
# ===================================================================
"""
Demostración del proceso completo:
1. Inicializar FeatureVisualizer
2. Generar imagen sintética
3. Visualizar resultado
"""

print("="*70)
print("🎨 EJEMPLO BÁSICO: Generar Feature de una Neurona")
print("="*70)

# ===================================================================
# CONFIGURACIÓN
# ===================================================================

# Capa a visualizar
# AlexNet: 'features.0', 'features.3', 'features.6', 'features.8', 'features.10'
TARGET_LAYER = 'features.0'

# ===================================================================
# PASO 1: Inicializar visualizador
# ===================================================================

visualizer = FeatureVisualizer(
    model=model,
    target_layer=TARGET_LAYER,
    device=device,
    input_size=(224, 224)
)

In [ ]:
# ===================================================================
# 2.2 EJECUTAR BARRIDO CON CACHÉ
# ===================================================================

import json
from pathlib import Path

print("="*70)
print("🔬 BARRIDO DE NEURONAS ACTIVAS (Con caché)")
print("="*70)

# Configuración
TOP_K = 15
CACHE_DIR = Path('../data/neuron_cache')
CACHE_DIR.mkdir(parents=True, exist_ok=True)

# Nombre del archivo cache basado en capa y modelo
cache_file = CACHE_DIR / f"{model.__class__.__name__}_{TARGET_LAYER.replace('.', '_')}.json"

# ===================================================================
# Intentar cargar desde cache
# ===================================================================

if cache_file.exists():
    print(f"📂 Cache encontrado: {cache_file.name}")
    with open(cache_file, 'r') as f:
        cache_data = json.load(f)
    
    active_neurons = [(n['neuron_idx'], n['activation']) for n in cache_data['neurons'][:TOP_K]]
    
    print(f"✅ Cargado desde cache:")
    print(f"   Capa: {cache_data['layer']}")
    print(f"   Total neuronas analizadas: {cache_data['total_neurons']}")
    print(f"   Fecha: {cache_data['timestamp']}")

else:
    # ===================================================================
    # No hay cache: ejecutar barrido
    # ===================================================================
    
    print(f"⚠️  No hay cache, ejecutando barrido completo...")
    
    active_neurons = find_active_neurons(
        model=model,
        target_layer=TARGET_LAYER,
        device=device,
        top_k=256,  # Guardar TODAS las neuronas
        test_iterations=50,
        visualizer=visualizer
    )
    
    # Guardar en cache
    from datetime import datetime
    
    cache_data = {
        'model': model.__class__.__name__,
        'layer': TARGET_LAYER,
        'total_neurons': len(active_neurons),
        'timestamp': datetime.now().isoformat(),
        'test_iterations': 50,
        'neurons': [
            {'neuron_idx': idx, 'activation': float(act)}
            for idx, act in active_neurons
        ]
    }
    
    with open(cache_file, 'w') as f:
        json.dump(cache_data, f, indent=2)
    
    print(f"💾 Cache guardado: {cache_file.name}")
    
    # Limitar a TOP_K para mostrar
    active_neurons = active_neurons[:TOP_K]

# ===================================================================
# Mostrar resultados
# ===================================================================

print(f"\n{'='*70}")
print(f"✅ TOP {TOP_K} NEURONAS MÁS ACTIVAS en {TARGET_LAYER}")
print(f"{'='*70}\n")

for rank, (neuron_idx, activation) in enumerate(active_neurons, 1):
    print(f"  {rank:2d}. Neurona {neuron_idx:3d} → Activación: {activation:8.4f}")

print(f"\n{'='*70}")
print(f"💡 Usar estas neuronas en la siguiente celda:")
print(f"   NEURON_IDX = {active_neurons[4][0]}  # Posición 5 (más interesante)")
print(f"{'='*70}\n")

In [ ]:
# ===================================================================
# CELDA 2.3: Ejemplo Básico - Generar Feature de una Neurona
# ===================================================================
"""
Demostración del proceso completo:
2. Generar imagen sintética
3. Visualizar resultado
"""
visualizer.cleanup()  # Limpiar hooks viejos
# Recrear
visualizer = FeatureVisualizer(  
    model=model,
    target_layer=TARGET_LAYER,
    device=device,
    input_size=(224, 224)
)
print("="*70)
print("🎨 EJEMPLO BÁSICO: Generar Feature de una Neurona")
print("="*70)

# Neurona específica
NEURON_IDX = 38

# Hiperparámetros de optimización
ITERATIONS = 500      # Más iteraciones = mejor convergencia
LEARNING_RATE = 0.1   # LR alto = convergencia rápida pero inestable
L2_DECAY = 1e-4       # Regularización L2
TV_WEIGHT = 1e-2      # Total Variation (suavizado) 1e-2 | 5e-2 para imágenes más suaves.


# ===================================================================
# PASO 2: Generar feature
# Capas profundas (conv4, conv5)
# Jitter: 2-4 píxeles
# Rotation: 1-5 grados
# Scale: 0.95-1.05 (±5%)
# ===================================================================

synthetic_img, history = visualizer.generate_feature(
    neuron_idx=NEURON_IDX,
    iterations=ITERATIONS,
    lr=LEARNING_RATE,
    l2_decay=L2_DECAY,
    tv_weight=TV_WEIGHT,
    jitter=4,              # Traslación aleatoria ±4 píxeles | 4-8 píxeles
    rotation_range=5.0,    # Rotación ±5 grados |  3-10 grados
    scale_range=(0.95, 1.05),  # Escala 95%-105% | 0.9-1.1 (±10%)
    blur_freq=4,           # Blur cada 4 iteraciones
    verbose=True
)

# ===================================================================
# PASO 3: Visualizar resultado
# ===================================================================

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Imagen generada
axes[0].imshow(synthetic_img)
axes[0].set_title(
    f'Feature Sintética\nNeurona {NEURON_IDX} - {TARGET_LAYER}',
    fontsize=12,
    fontweight='bold'
)
axes[0].axis('off')

# Curva de activación
axes[1].plot(history['activation'], linewidth=2, color='#2ecc71')
axes[1].fill_between(
    range(len(history['activation'])),
    history['activation'],
    alpha=0.3,
    color='#2ecc71'
)
axes[1].set_xlabel('Iteración', fontsize=11)
axes[1].set_ylabel('Activación', fontsize=11)
axes[1].set_title('Convergencia', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 INTERPRETACIÓN:")
print(f"   La imagen muestra qué patrón maximiza la neurona {NEURON_IDX}")
print(f"   Activación final: {history['activation'][-1]:.4f}")

# Cleanup
visualizer.cleanup()

In [ ]:
# ===================================================================
# CELDA 3: Análisis Detallado de Convergencia
# ===================================================================
"""
Visualiza todas las métricas de optimización:
- Activación de neurona
- Pérdida L2
- Total Variation
- Pérdida total
"""

print("="*70)
print("📊 ANÁLISIS DE CONVERGENCIA")
print("="*70)

# Usar el history de la celda anterior
plot_optimization_progress(history, NEURON_IDX)

print("\n💡 OBSERVACIONES:")
print("\n1️⃣ ACTIVACIÓN (verde):")
print("   - Debe SUBIR consistentemente")
print("   - Si oscila mucho → reducir learning rate")
print("   - Si se estanca → más iteraciones o ajustar regularización")

print("\n2️⃣ L2 LOSS (rojo):")
print("   - Penaliza píxeles con valores extremos")
print("   - Debe estabilizarse en valor bajo")
print("   - Si sube mucho → la imagen tiene valores demasiado altos")

print("\n3️⃣ TOTAL VARIATION (azul):")
print("   - Mide 'rugosidad' de la imagen")
print("   - TV bajo = imagen suave")
print("   - TV alto = imagen ruidosa")

print("\n4️⃣ TOTAL LOSS (morado):")
print("   - Combinación de todas las pérdidas")
print("   - Debe BAJAR (estamos minimizando)")
print("   - Nota: Activación se maximiza, pero loss = -activación")

print("\n" + "="*70)
print("🔧 GUÍA DE AJUSTE DE HIPERPARÁMETROS")
print("="*70)

print("\n📊 DIAGNÓSTICO según tu gráfico:")
print("   ✅ Activación sube → Optimización funciona")
print("   ✅ L2 controlado → Píxeles no saturan")
print("   ⚠️  TV sube ligeramente → Imagen tiene algo de ruido")
print("   ✅ Total Loss baja → Convergencia correcta")

print("\n⚙️  AJUSTES RECOMENDADOS:")

print("\n🎯 Si ACTIVACIÓN no sube:")
print("   → ITERATIONS: 500 → 800")
print("   → LEARNING_RATE: 0.1 → 0.15")
print("   → Probar otra neurona (puede estar 'muerta')")

print("\n🎨 Si imagen sale MUY RUIDOSA (TV alto):")
print("   → TV_WEIGHT: 1e-2 → 5e-2")
print("   → Más blur: blur_freq: 4 → 2")

print("\n💥 Si imagen sale SATURADA (colores extremos):")
print("   → L2_DECAY: 1e-4 → 5e-4")
print("   → LEARNING_RATE: 0.1 → 0.05")

print("\n🐌 Si converge MUY LENTO:")
print("   → LEARNING_RATE: 0.1 → 0.2")
print("   → L2_DECAY: 1e-4 → 5e-5 (menos restricción)")

print("\n🎲 Si oscila mucho (inestable):")
print("   → LEARNING_RATE: 0.1 → 0.05")
print("   → Más iteraciones para compensar")

print("\n" + "="*70)
print("💡 TU CASO ESPECÍFICO (según gráfico):")
print("="*70)
print("   Estado: ✅ Buena optimización")
print("   Sugerencia: TV puede mejorar levemente")
print("   Acción: TV_WEIGHT = 2e-2 (duplicar)")
print("\n" + "="*70)

In [ ]:
# ===================================================================
# CELDA 3: Grid de Features - Múltiples Neuronas
# ===================================================================
"""
Genera y visualiza features de múltiples neuronas simultáneamente.
Útil para comparar qué detecta cada filtro.
"""

print("="*70)
print("🎨 GRID DE FEATURES - Múltiples Neuronas")
print("="*70)

# ===================================================================
# CONFIGURACIÓN
# ===================================================================


# Seleccionar neuronas a visualizar
# Estrategia 1: Primeras N neuronas
# TARGET_LAYER_GRID = 'features.0'
# NEURON_INDICES = list(range(12))  # [0, 1, 2, ..., 11]

# Estrategia 2: Neuronas del ranking de activación
TARGET_LAYER_GRID = TARGET_LAYER
NEURON_INDICES = [n[0] for n in active_neurons[:12]]  # Top 12 del ranking

# Parámetros de optimización (reducidos para velocidad)
GRID_ITERATIONS = 300
GRID_LR = 0.1

# ===================================================================
# GENERACIÓN
# ===================================================================

visualizer_grid = FeatureVisualizer(
    model=model,
    target_layer=TARGET_LAYER_GRID,
    device=device
)

images, histories = visualizer_grid.generate_grid(
    neuron_indices=NEURON_INDICES,
    iterations=GRID_ITERATIONS,
    lr=GRID_LR,
    verbose=False  # Silenciar logs individuales
)

# ===================================================================
# VISUALIZACIÓN
# ===================================================================

plot_feature_grid(
    images=images,
    neuron_indices=NEURON_INDICES,
    layer_name=TARGET_LAYER_GRID,
    ncols=4,
    figsize=(16, 12)
)

# Curvas de convergencia
plot_convergence(
    histories=histories[:6],  # Mostrar solo primeras 6 para claridad
    neuron_indices=NEURON_INDICES[:6]
)

# Limpio el TARGET_LAYER_GRID y obtengo el .N para saber que tan profunda es la capa

# Si la capa es menor de 3
if int(TARGET_LAYER_GRID.split('.')[-1]) < 3:

    print("\n💡 INTERPRETACIÓN:")
    print("\n🔍 En capas TEMPRANAS (como features.0 en AlexNet):")
    print("   - Detectores de BORDES (líneas, contornos)")
    print("   - Detectores de COLOR (canales RGB específicos)")
    print("   - Detectores de TEXTURAS simples (rayas, puntos)")
    print("   - Patrones GEOMÉTRICOS básicos")

else:

    print("\n💡 INTERPRETACIÓN:")
    print("\n🔬 En capas PROFUNDAS:")
    print("   - Patrones MÁS COMPLEJOS y abstractos")
    print("   - Combinaciones de features de capas anteriores")
    print("   - Más difíciles de interpretar visualmente")

print("\n✅ Experimenta cambiando TARGET_LAYER_GRID")
print("   y observa cómo cambian los patrones!")

# Cleanup
visualizer_grid.cleanup()

In [ ]:
# ===================================================================
# CELDA 5.1: Preparar Imagen Real
# ===================================================================

from pathlib import Path
import requests
from io import BytesIO
from PIL import Image

print("="*70)
print("📥 PREPARAR IMAGEN REAL")
print("="*70)

# Configuración
data_dir = Path("../data/test_images")
data_dir.mkdir(parents=True, exist_ok=True)

# Opción 1: Descargar imagen de ejemplo
image_url = "https://images.unsplash.com/photo-1574158622682-e40e69881006?w=400"
image_path = data_dir / "cat_example.jpg"

if not image_path.exists():
    print("📥 Descargando imagen de ejemplo...")
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    img.save(image_path)
    print(f"✅ Guardada en: {image_path}")
else:
    print(f"✅ Imagen ya existe: {image_path}")

# Opción 2: Usar tu propia imagen (descomentar)
# image_path = Path("ruta/a/tu/imagen.jpg")

print(f"\n🖼️  Imagen seleccionada: {image_path.name}")

In [ ]:
# ===================================================================
# CELDA 5.2: Comparación Real vs Sintética
# ===================================================================

print("="*70)
print("🔍 REAL vs SINTÉTICA: Comparación de Activaciones")
print("="*70)

# Configuración
COMPARE_LAYER = TARGET_LAYER
COMPARE_NEURON = NEURON_IDX  # Usar la neurona actual
COMPARE_ITERATIONS = 500

print(f"\n🎯 Analizando:")
print(f"   Capa: {COMPARE_LAYER}")
print(f"   Neurona: {COMPARE_NEURON}")

# Ejecutar comparación
comparison = compare_real_vs_synthetic(
    model=model,
    target_layer=COMPARE_LAYER,
    device=device,
    neuron_idx=COMPARE_NEURON,
    real_image_path=str(image_path),
    iterations=COMPARE_ITERATIONS
)

# Visualizar comparación básica
plot_real_vs_synthetic(comparison)

print(f"\n📊 Resultados:")
print(f"   Activación Real: {comparison['real_activation']:.4f}")
print(f"   Activación Sintética: {comparison['synthetic_activation']:.4f}")
print(f"   Mejora: {comparison['improvement']:.2f}x")

In [ ]:
# ===================================================================
# CELDA 5.3: Localizar Zona de Máxima Activación
# ===================================================================
from src.utils.visualization_helpers import extract_max_activation_region, plot_activation_heatmap_with_roi
print("="*70)
print("📍 LOCALIZAR ZONA DE MÁXIMA ACTIVACIÓN")
print("="*70)

# Extraer información de activación espacial
activation_info = extract_max_activation_region(
    model=model,
    target_layer=COMPARE_LAYER,
    neuron_idx=COMPARE_NEURON,
    image_path=str(image_path),
    device=device
)

print(f"\n📐 Información de activación:")
print(f"   Posición máxima: ({activation_info['center_x']}, {activation_info['center_y']})")
print(f"   Valor máximo: {activation_info['max_value']:.4f}")
print(f"   Tamaño mapa: {activation_info['map_shape']}")

# Visualizar heatmap con ROI
plot_activation_heatmap_with_roi(
    real_image=activation_info['real_image'],
    activation_map=activation_info['activation_map'],
    center_x=activation_info['center_x'],
    center_y=activation_info['center_y'],
    patch_size=activation_info['patch_size'],
    neuron_idx=COMPARE_NEURON,
    layer_name=COMPARE_LAYER
)

In [ ]:
# ===================================================================
# CELDA 5.4: Superposición Real + Sintética
# ===================================================================

from src.utils.visualization_helpers import overlay_synthetic_on_activation, plot_overlay_comparison


print("="*70)
print("🎯 SUPERPOSICIÓN: Patrón Real vs Ideal")
print("="*70)

# Extraer parche y superponer con sintética
overlay_result = overlay_synthetic_on_activation(
    real_image=activation_info['real_image'],
    synthetic_image=comparison['synthetic_image'],
    center_x=activation_info['center_x'],
    center_y=activation_info['center_y'],
    patch_size=activation_info['patch_size']
)

# Visualizar superposición
plot_overlay_comparison(
    real_full=overlay_result['real_full'],
    real_patch=overlay_result['real_patch'],
    synthetic_resized=overlay_result['synthetic_resized'],
    blend=overlay_result['blend'],
    center_x=activation_info['center_x'],
    center_y=activation_info['center_y'],
    patch_size=activation_info['patch_size'],
    neuron_idx=COMPARE_NEURON,
    layer_name=COMPARE_LAYER
)

print("\n💡 INTERPRETACIÓN:")
print("   - Cuadro rojo: Región de máxima activación")
print("   - Parche real: Qué ve la neurona en la imagen")
print("   - Sintética: Patrón ideal que busca")
print("   - Blend: Coincidencias entre real e ideal")

In [ ]:
# ===================================================================
# CELDA 5.5: Análisis Final
# ===================================================================

print("="*70)
print("🎓 ANÁLISIS Y CONCLUSIONES")
print("="*70)

print(f"\n📊 MÉTRICAS:")
print(f"   Activación Real: {comparison['real_activation']:.4f}")
print(f"   Activación Sintética: {comparison['synthetic_activation']:.4f}")
print(f"   Mejora: {comparison['improvement']:.2f}x")
print(f"   Zona activa: ({activation_info['center_x']}, {activation_info['center_y']})")

print("\n🎓 LECCIONES:")

print("\n1️⃣ Si sintética >> real (mejora > 10x):")
print("   → La neurona busca patrones MUY ESPECÍFICOS")
print("   → La imagen real no contiene esos patrones ideales")
print("   → La sintética muestra el 'concepto puro'")

print("\n2️⃣ Si sintética ≈ real (mejora < 3x):")
print("   → La imagen real ya tiene los patrones buscados")
print("   → Difícil mejorar sin cambiar estructura")
print("   → La neurona está bien representada")

print("\n3️⃣ Diferencias visuales:")
print("   → Sintética: Patrón PURO sin contexto")
print("   → Real: Patrón en contexto natural")
print("   → Blend: Muestra qué características coinciden")

print("\n🔍 EN TU CASO:")
improvement = comparison['improvement']
if improvement > 10:
    print(f"   ✅ Mejora alta ({improvement:.1f}x) → Patrón muy específico")
    print("      La sintética revela qué busca idealmente")
elif improvement > 3:
    print(f"   ⚙️  Mejora moderada ({improvement:.1f}x) → Patrón parcial")
    print("      Coincidencia parcial entre real e ideal")
else:
    print(f"   ⚠️  Mejora baja ({improvement:.1f}x) → Patrón presente")
    print("      La imagen real ya tiene el patrón")

print("\n" + "="*70)

In [ ]:
# ===================================================================
# CELDA 6: Comparación entre Capas
# ===================================================================
"""
Genera features de la MISMA neurona en DIFERENTES capas.
Observa cómo aumenta la complejidad con la profundidad.
"""

print("="*70)
print("🔬 COMPARACIÓN ENTRE CAPAS - Evolución de Features")
print("="*70)

# ===================================================================
# CONFIGURACIÓN
# ===================================================================

# Capas a comparar (AlexNet)
LAYERS_TO_COMPARE = [
    'features.0',   # Conv1: 11x11, stride 4
    'features.3',   # Conv2: 5x5
    'features.6',   # Conv3: 3x3
    'features.8',   # Conv4: 3x3
]

# Para ResNet18, usar:
# LAYERS_TO_COMPARE = [
#     'conv1',
#     'layer1.0.conv1',
#     'layer2.0.conv1',
#     'layer3.0.conv1'
# ]

NEURON_TO_TRACK = 5  # Misma neurona en todas las capas
LAYER_ITERATIONS = 400

# ===================================================================
# GENERAR FEATURES
# ===================================================================

layer_images = []
layer_names = []

for layer_name in LAYERS_TO_COMPARE:
    print(f"\n📍 Procesando: {layer_name}")
    
    vis = FeatureVisualizer(
        model=model,
        target_layer=layer_name,
        device=device
    )
    
    img, hist = vis.generate_feature(
        neuron_idx=NEURON_TO_TRACK,
        iterations=LAYER_ITERATIONS,
        lr=0.1,
        verbose=False
    )
    
    layer_images.append(img)
    layer_names.append(layer_name)
    
    vis.cleanup()

# ===================================================================
# VISUALIZACIÓN
# ===================================================================

fig, axes = plt.subplots(1, len(layer_images), figsize=(18, 5))

for ax, img, name in zip(axes, layer_images, layer_names):
    ax.imshow(img)
    ax.set_title(
        f'{name}\nNeurona {NEURON_TO_TRACK}',
        fontsize=11,
        fontweight='bold'
    )
    ax.axis('off')

plt.suptitle(
    f'Evolución de Features - Neurona {NEURON_TO_TRACK} en Diferentes Capas',
    fontsize=14,
    fontweight='bold',
    y=1.02
)

plt.tight_layout()
plt.show()

print("\n💡 OBSERVACIONES ESPERADAS:")
print("\n📊 CAPA 1 (features.0):")
print("   - Patrones SIMPLES: bordes, líneas, colores")
print("   - MUY interpretables visualmente")
print("   - Campo receptivo PEQUEÑO (11x11)")

print("\n📊 CAPAS INTERMEDIAS (features.3-6):")
print("   - Patrones MÁS COMPLEJOS")
print("   - Combinaciones de bordes y texturas")
print("   - Empiezan a aparecer formas")

print("\n📊 CAPAS PROFUNDAS (features.8+):")
print("   - Patrones ABSTRACTOS")
print("   - Difíciles de describir verbalmente")
print("   - Representaciones de ALTO NIVEL")
print("   - Campo receptivo MUY GRANDE (cubre casi toda la imagen)")

print("\n🎯 CONCLUSIÓN:")
print("   Las redes procesan información JERÁRQUICAMENTE:")
print("   Píxeles → Bordes → Texturas → Partes → Objetos")

print("\n" + "="*70)

In [ ]:
# ===================================================================
# CELDA 7: Experimentación con Hiperparámetros
# ===================================================================
"""
Compara cómo diferentes hiperparámetros afectan las features generadas.
Experimento: Variar regularización L2 y TV.
"""

print("="*70)
print("🧪 EXPERIMENTO: Impacto de Regularización")
print("="*70)

# ===================================================================
# CONFIGURACIÓN DEL EXPERIMENTO
# ===================================================================

EXPERIMENT_LAYER = TARGET_LAYER
EXPERIMENT_NEURON = NEURON_IDX
EXPERIMENT_ITERATIONS = 400

# Configuraciones a probar
configs = [
    {'name': 'Sin regularización', 'l2': 0, 'tv': 0},
    {'name': 'Solo L2', 'l2': 1e-3, 'tv': 0},
    {'name': 'Solo TV', 'l2': 0, 'tv': 1e-1},
    {'name': 'Ambas (balanceado)', 'l2': 1e-4, 'tv': 1e-2},
]

# ===================================================================
# EJECUTAR EXPERIMENTO
# ===================================================================

results = []

vis_exp = FeatureVisualizer(
    model=model,
    target_layer=EXPERIMENT_LAYER,
    device=device
)

for config in configs:
    print(f"\n🔬 Probando: {config['name']}")
    print(f"   L2: {config['l2']}, TV: {config['tv']}")
    
    img, hist = vis_exp.generate_feature(
        neuron_idx=EXPERIMENT_NEURON,
        iterations=EXPERIMENT_ITERATIONS,
        lr=0.1,
        l2_decay=config['l2'],
        tv_weight=config['tv'],
        verbose=False
    )
    
    results.append({
        'config': config,
        'image': img,
        'history': hist
    })

vis_exp.cleanup()

# ===================================================================
# VISUALIZACIÓN
# ===================================================================

fig, axes = plt.subplots(2, 4, figsize=(18, 10))

# Fila 1: Imágenes
for i, result in enumerate(results):
    axes[0, i].imshow(result['image'])
    axes[0, i].set_title(
        result['config']['name'],
        fontsize=10,
        fontweight='bold'
    )
    axes[0, i].axis('off')

# Fila 2: Curvas de activación
for i, result in enumerate(results):
    axes[1, i].plot(
        result['history']['activation'],
        linewidth=2,
        color='#3498db'
    )
    axes[1, i].set_title('Activación', fontsize=9)
    axes[1, i].set_xlabel('Iteración', fontsize=8)
    axes[1, i].grid(True, alpha=0.3)
    
    # Añadir valor final
    final_act = result['history']['activation'][-1]
    axes[1, i].text(
        0.5, 0.95,
        f'Final: {final_act:.3f}',
        transform=axes[1, i].transAxes,
        fontsize=9,
        verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    )

plt.suptitle(
    f'Impacto de Regularización - Neurona {EXPERIMENT_NEURON}',
    fontsize=14,
    fontweight='bold',
    y=0.98
)

plt.tight_layout()
plt.show()

print("\n📊 ANÁLISIS DE RESULTADOS:")
print("\n1️⃣ SIN REGULARIZACIÓN:")
print("   ❌ Imagen muy ruidosa (adversarial)")
print("   ❌ Difícil de interpretar")
print("   ✅ Activación MUY alta (sobreajuste a píxeles)")

print("\n2️⃣ SOLO L2:")
print("   ✓ Reduce valores extremos")
print("   ⚠️  Puede seguir siendo ruidosa")
print("   ✓ Activación alta pero controlada")

print("\n3️⃣ SOLO TV:")
print("   ✅ Imagen MUY suave")
print("   ✅ Más interpretable")
print("   ⚠️  Puede perder detalles")

print("\n4️⃣ AMBAS (RECOMENDADO):")
print("   ✅ Balance entre activación y naturalidad")
print("   ✅ Imágenes interpretables")
print("   ✅ Captura la esencia del patrón")

print("\n" + "="*70)

## 🎯 Tips y Mejores Prácticas

### Para mejores resultados:

1. **Capas tempranas** (conv1):
   - Menos iteraciones (200-300)
   - LR más bajo (0.05-0.1)
   - Convergen rápido

2. **Capas profundas**:
   - Más iteraciones (500-1000)
   - LR más alto (0.1-0.2)
   - Necesitan más optimización

3. **Regularización**:
   - `l2_decay=1e-4` → Punto de partida estándar
   - `tv_weight=1e-2` → Ajustar según suavidad deseada
   - TV alto → Imágenes más borrosas pero limpias
   - TV bajo → Más detalle pero puede ser ruidoso

4. **Transformaciones**:
   - Jitter: Siempre útil (4-8 píxeles)
   - Rotación: Opcional para patrones no orientados
   - Escala: Útil para multi-escala

### Troubleshooting:

- **Imagen muy ruidosa** → Aumentar `tv_weight`
- **Activación no sube** → Aumentar LR o más iteraciones
- **Oscila mucho** → Reducir LR
- **Demasiado borrosa** → Reducir `tv_weight`
- **Colores saturados** → Aumentar `l2_decay`

---

In [ ]:
# ===================================================================
# CELDA 8: Guardar Resultados (OPCIONAL)
# ===================================================================
"""
Guarda las features generadas en disco para análisis posterior.
"""

from src.utils.visualization_helpers import save_feature_collection

# Configuración
SAVE_DIR = "../outputs/features"
SAVE = False  # Cambiar a True para guardar

if SAVE and 'images' in locals():
    save_feature_collection(
        images=images,
        neuron_indices=NEURON_INDICES,
        save_dir=SAVE_DIR,
        layer_name=TARGET_LAYER_GRID.replace('.', '_'),
        prefix="feature"
    )
else:
    print("💡 Para guardar, ejecuta la celda 4 (Grid) y cambia SAVE=True")

## 🎓 Conclusiones

### Lo que aprendimos:

1. **Feature Visualization** nos permite ver qué "buscan" las neuronas
2. **Capas tempranas** detectan patrones simples (bordes, colores)
3. **Capas profundas** detectan patrones complejos y abstractos
4. **Regularización** es crucial para imágenes interpretables
5. Las CNNs procesan información **jerárquicamente**

### Limitaciones:

- Solo vemos **activación promedio**, no dónde activa
- Capas profundas son **difíciles de interpretar** visualmente
- Las features sintéticas pueden **no ser naturales**
- No capturamos **interacciones entre neuronas**

### Próximos pasos:

- **Activation Maximization** con restricciones más fuertes
- **DeepDream**: Amplificar patrones en imágenes reales
- **Neural Style Transfer**: Combinar contenido y estilo
- **GAN-based visualization**: Usar GANs pre-entrenadas

---

## 📚 Referencias

- Olah et al. (2017) - "Feature Visualization"
- Mordvintsev et al. (2015) - "Inceptionism: Going Deeper into Neural Networks" (DeepDream)
- Nguyen et al. (2016) - "Synthesizing the preferred inputs for neurons in neural networks"
- Distill.pub - Feature Visualization series

---